In [1]:
import os
import pandas as pd
rating_file_path= 'C:/Users/minho/aiffel/datasets/ml-1m/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

user_id  movie_id  rating  timestamp
0        1      1193       5  978300760
1        1       661       3  978302109
2        1       914       3  978301968
3        1      3408       4  978300275
4        1      2355       5  978824291

In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [6]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [7]:
ratings['user_id'].nunique()
# rating에 있는 유니크한 사용자 수

6039

In [8]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path= 'C:/Users/minho/aiffel/datasets/ml-1m/ml-1m/ratings.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')

In [9]:
movies['movie_id'].nunique()
# ratings에 있는 유니크한 영화 개수

3706

In [10]:
movies_file_path = 'C:/Users/minho/aiffel/datasets/ml-1m/ml-1m/movies.dat'
movies_file_path
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
movies = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
movies

user_id  movie_id  rating  timestamp
0              1      1193       5  978300760
1              1       661       3  978302109
2              1       914       3  978301968
3              1      3408       4  978300275
4              1      2355       5  978824291
...          ...       ...     ...        ...
1000204     6040      1091       1  956716541
1000205     6040      1094       5  956704887
1000206     6040       562       5  956704746
1000207     6040      1096       4  956715648
1000208     6040      1097       4  956715569

[1000209 rows x 4 columns]

In [11]:
movies_count = movies.groupby('movie_id')['rating'].count()
movies_count.sort_values(ascending=False).head(30)
# 가장 인기있는 영화 30개(인기순)

movie_id
2858    3428
260     2991
1196    2990
1210    2883
480     2672
2028    2653
589     2649
2571    2590
1270    2583
593     2578
1580    2538
1198    2514
608     2513
2762    2459
110     2443
2396    2369
1197    2318
527     2304
1617    2288
1265    2278
1097    2269
2628    2250
2997    2241
318     2227
858     2223
356     2194
2716    2181
296     2171
1240    2098
1       2077
Name: rating, dtype: int64

In [12]:
user_count = movies.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6040.000000
mean      165.597517
std       192.747029
min        20.000000
25%        44.000000
50%        96.000000
75%       208.000000
max      2314.000000
Name: movie_id, dtype: float64

In [13]:
user_median = movies.groupby('user_id')['movie_id'].median()
user_median.describe()

count    6040.000000
mean     1846.879139
std       514.532830
min       597.000000
25%      1466.000000
50%      1733.500000
75%      2052.000000
max      3919.000000
Name: movie_id, dtype: float64

In [17]:
# my_favourite = ['love rose', 'avengers', 'Fantastic beasts and where to find them', 'doctor dolittle', 'kingsman']
# 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.
my_favourite = [7000,7001,7002,7003,7005]

import datetime
a = datetime.datetime.now()
my_rate = pd.DataFrame({'user_id': [6041]*5, 'movie_id': my_favourite, 'rating': [5]*5, 'timestamp': [a]*5, 'count': [5.0]*5})
if not ratings.isin({'user_id':[6041]})['user_id'].any():
    ratings = ratings.append(my_rate)
ratings.tail(10)

user_id  movie_id  count                   timestamp  rating
1000203     6040      1090    3.0                   956715518     NaN
1000205     6040      1094    5.0                   956704887     NaN
1000206     6040       562    5.0                   956704746     NaN
1000207     6040      1096    4.0                   956715648     NaN
1000208     6040      1097    4.0                   956715569     NaN
0           6041      7000    5.0  2020-10-31 17:11:14.211482     5.0
1           6041      7001    5.0  2020-10-31 17:11:14.211482     5.0
2           6041      7002    5.0  2020-10-31 17:11:14.211482     5.0
3           6041      7003    5.0  2020-10-31 17:11:14.211482     5.0
4           6041      7005    5.0  2020-10-31 17:11:14.211482     5.0

In [19]:
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [20]:
print(user_to_idx[6041])
print(movie_to_idx[7000])

6039
3628


In [29]:
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):
    print('user column indexing ok')
    ratings['user_id'] = temp_user_data
else:
    print('user column indexing failed')

temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing ok')
    
else: 
    print('movie column indexing failed')

ratings

user column indexing ok
movie column indexing ok


user_id  movie_id  count                   timestamp  rating
0         0      1193    5.0                   978300760     NaN
1         0       661    3.0                   978302109     NaN
2         0       914    3.0                   978301968     NaN
3         0      3408    4.0                   978300275     NaN
4         0      2355    5.0                   978824291     NaN
..      ...       ...    ...                         ...     ...
0      6039      7000    5.0  2020-10-31 17:11:14.211482     5.0
1      6039      7001    5.0  2020-10-31 17:11:14.211482     5.0
2      6039      7002    5.0  2020-10-31 17:11:14.211482     5.0
3      6039      7003    5.0  2020-10-31 17:11:14.211482     5.0
4      6039      7005    5.0  2020-10-31 17:11:14.211482     5.0

[836483 rows x 5 columns]

In [33]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings.count, (ratings.movie_id, ratings.user_id)), shape = (num_user, num_movie))
csr_data

TypeError: len() of unsized object